In [ ]:
from gurobipy import Model, GRB

# 모델 생성
model = Model("EV_Optimization")
model.setParam('NonConvex', 2)
# 시간 설정
T = 24  # 24시간 시뮬레이션


#  최대 태양광 발전량 (PV Generation)
pv_generation = [0] * 5 + [200, 500, 1000, 1500, 1800, 2000, 1800, 1500, 1000, 500, 200] + [0] * 8
# 최대 송전량량
transmission_power=5000

# 전력당 비용 
cost_pv =[1,2,3]
cost_tp =[1,2,3]

# 100프로 충전 되지 않은 것에 대한 불편의 가중치중치
penalty_charge=10

# 충전소 관련 조건
p_cs_max=[200,200,200]
p_cs_min=[0,0,0]
wpref_s, wpref_f, L_WM = 6, 22, 2  # 세탁기 사용 시간 (06:00 - 22:00), 연속 2시간

# 전기차 관련 조건
SOE_init, SOE_min, SOE_max = 2400, 800, 4000  # ESS 초기, 최소, 최대 에너지 저장량
DE_AC, DE_ESS = 40, 150  # 에너지 소비 단위 (Wh
wpref_s, wpref_f, L_WM = 6, 22, 2  # 세탁기 사용 시간 (06:00 - 22:00), 연속 2시간


E_max_AC, E_max_WM, E_max_UC = 3000, 500, 1700  # 최대 소비 전력 (Wh)
Ech_max, Edch_max = 4000, 4000  # ESS 충/방전 최대 용량



# 변수 선언
P_pv= model.addVars(T, vtype=GRB.CONTINUOUS, lb=0, ub=pv_generation[T], name="P_pv")
P_tp= model.addVars(T, vtype=GRB.CONTINUOUS, lb=0, ub=transmission_power, name="P_tv")

Charge_state_ev = model.addVars(T, vtype=GRB.BINARY, name="Charge_state_ev0") # 충전 중 이진수

delta_T = model.addVars(T, vtype=GRB.CONTINUOUS, name="delta_T")
T_in = model.addVars(T+1, vtype=GRB.CONTINUOUS, name="T_in")
E_AC = model.addVars(T, vtype=GRB.CONTINUOUS, lb=0, ub=E_max_AC, name="E_AC")
E_WM = model.addVars(T, vtype=GRB.BINARY, name="E_WM")  # 세탁기는 이진 변수 (0 또는 1)
SOE = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=SOE_min, ub=SOE_max, name="SOE")
E_ESS_ch = model.addVars(T, vtype=GRB.CONTINUOUS, name="E_ESS_ch")
E_ESS_dch = model.addVars(T, vtype=GRB.CONTINUOUS, name="E_ESS_dch")
E_net = model.addVars(T, vtype=GRB.CONTINUOUS, name="E_net")



In [ ]:
for t in range(T):
    if t < wpref_s or t >= wpref_f:
        model.addConstr(E_WM[t] == 0)  # 선호 시간대 외에는 작동 불가

# 세탁기 총 작동 시간 제약
model.addConstr(sum(E_WM[t] for t in range(T)) == L_WM)

# 연속 작동 보장을 위한 제약 - 수정
for p in range(wpref_s, wpref_f - L_WM + 1):
    model.addConstr(sum(E_WM[t] for t in range(p, p + L_WM)) >= (E_WM[p] - E_WM[p-1]) * L_WM)

model.addConstr(T_in[0] == T_set)
model.addConstr(SOE[0] == SOE_init)
# 나머지 제약조건들
for t in range(T):
    #실내 온도 변화 모델 이게 문제
    if t > 0:
        model.addConstr(E_AC[t] - E_AC[t-1] <= DE_AC)
        model.addConstr(E_AC[t] - E_AC[t-1] >= -1*DE_AC)
    else:
        model.addConstr(T_in[t+1] == T_in[t+1] + beta * E_AC[t])
    
    # 선형화
    model.addConstr(delta_p[t] >= T_in[t] - T_set)
    model.addConstr(delta_p[t] >= T_set - T_in[t])
    
for t in range(T):
    # ESS 에너지 저장 모델
    model.addConstr(SOE[t+2] == SOE[t+1] + 0.9 * E_ESS_ch[t] - E_ESS_dch[t] / 0.9)
    model.addConstr(SOE[t+1] - SOE[t] <= DE_ESS)
    model.addConstr(SOE[t+1] - SOE[t] >= -1*DE_ESS)
model.addConstr(SOE[24] - SOE[23] <= DE_ESS)
model.addConstr(SOE[24] - SOE[23] >= -1*DE_ESS)


for t in range(T):
    # ESS 충/방전 제한
    model.addConstr(E_ESS_ch[t] >= 0 * E_charge[t])
    model.addConstr(E_ESS_ch[t] <= Ech_max * E_charge[t])
    model.addConstr(E_ESS_dch[t] >= 0 * (1-E_charge[t]))
    model.addConstr(E_ESS_dch[t] <= Edch_max * (1-E_charge[t]))

    # 순 전력 소비 모델 왜 이것만 들어가면 이상해지는 거지
    model.addConstr(E_AC[t] + E_WM[t] * E_max_WM + E_ESS_ch[t] - E_ESS_dch[t] - pv_generation[t] <= E_net[t])



In [2]:
a= [1,2,3]
b=[1,2,4]
c=sum(a[0]*b[t] + a[1]*b[t]for t in range(2))
print(c)

9


In [ ]:
# 목적함수 무조건 맞아
model.setObjective(
    sum(cost_pv[0] * P_pv[t]*P_pv[t]+ cost_pv[1] * P_pv[t]+ cost_pv[2] for t in range(T))+  # 전기 요금 비용
    sum(cost_tp[0] * P_tp[t]*P_tp[t]+ cost_tp[1] * P_tp[t]+ cost_tp[2] for t in range(T))+  # 전기 요금 비용
    penalty_charge * sum(delta_p[t] for t in range(T)),     # 충전 불편 내용용
    GRB.MINIMIZE
)

# 최적화 실행
model.optimize()

# 결과 출력
if model.status == GRB.OPTIMAL:
    print("\n최적화 성공!")
    print("\n=== 시간별 결과 ===")
    for t in range(T):
        print(f"Hour {t:2d}: delta_T = {delta_T[t].x:6.2f}°C, "
              f"E_AC = {E_AC[t].x:7.2f} Wh, "
              f"E_WM = {E_WM[t].x:1.0f}, "
              f"SOE = {SOE[t].x:7.2f} Wh")
    
    # 총 비용 계산
    total_cost = sum(E_net[t].x * tou_price[t] for t in range(T))
    print(f"\n총 전기 요금: {total_cost:.2f}")
    
    # 평균 실내 온도
    avg_temp = sum(T_in[t].x for t in range(T)) / T
    print(f"평균 실내 온도: {avg_temp:.2f}°C")
else:
    print("\n최적화 실패!")
    print(f"Status code: {model.status}")
    if model.status == GRB.INFEASIBLE:
        print("문제가 실행 불가능합니다. 제약조건을 확인해주세요.")
    elif model.status == GRB.UNBOUNDED:
        print("문제가 무한대 해를 가집니다. 제약조건을 확인해주세요.")
    else:
        print("다른 오류가 발생했습니다. Gurobi 상태 코드를 확인해주세요.")

In [ ]:
# 목적함수 설정: minimize y - x^2 + 3
model.setObjective(y + x*x + 3, GRB.MINIMIZE)

# 최적화 실행
model.optimize()

# 최적해 출력
if model.status == GRB.OPTIMAL:
    print("최적해:")
    print("x =", x.X)
    print("y =", y.X)
    print("최소 목적함수 값 =", model.objVal)
else:
    print("최적해를 찾지 못했습니다. 상태 코드:", model.status)